<img src="https://ucfai.org/core/fa19/cnns/banner.png">

<div class="col-12">
    <span class="btn btn-success btn-block">
        Meeting in-person? Have you signed in?
    </span>
</div>

<div class="col-12">
    <h1> How We Can Give Our Computers Eyes and Ears </h1>
    <hr>
</div>

<div style="line-height: 2em;">
    <p>by: 
        <strong> Daniel Silva</strong>
        (<a href="https://github.com/danielzgsilva">@danielzgsilva</a>)
    
        <strong> Brandon</strong>
        (<a href="https://github.com/brandons209">@brandons209</a>)
     on 2019-10-16</p>
</div>